In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # rank of LoRA
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none", # No bias added for optimal performance
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False, 
    loftq_config = None,
)

In [ ]:
import pandas as pd
from datasets import Dataset

In [ ]:
EOS_TOKEN = tokenizer.eos_token
df = pd.read_excel('train.xlsx')
prompt = "Summarize the provided code solution for the given problem in simple, plain English text. Explain in simple text how the code works to solve the specified problem."
df["question"] = (
    prompt
    + "\n Question: )"
    + df["Question"]
    + "\n Code: )"
    + df["Code"]
    + "\n Plain Text: )"
    + df["Plain Text"]
    + EOS_TOKEN
)

custom_ds = pd.DataFrame()
custom_ds["text"] = df["question"]
dataset = Dataset.from_pandas(custom_ds)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 15,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
test_df = pd.read_excel('test.xlsx')
test_row = test_df.head(1)
test_text = prompt + "\n Question: )" + test_row["Question"].values[0] + "\n Code: )" + test_row["Code"].values[0] + "\n Plain Text: )"
print(test_text)

In [ ]:
FastLanguageModel.for_inference(model)

inputs = tokenizer(
    [test_text],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=200)

# Decode the generated tokens, excluding the input tokens
generated_sequence = outputs[0]
input_length = inputs['input_ids'].shape[1]  # Length of the input text tokens
new_tokens = generated_sequence[input_length:]  # Exclude the input tokens

new_text = tokenizer.decode(new_tokens, skip_special_tokens=True)

print(new_text)